## 1. Before you start

- [ ] connect to [SSPCloud](https://datalab.sspcloud.fr/)
  - if necessary, create an account with your ENSAI e-mail address
- Go to `Mes services`
  - [ ] Create a new service : `Jupyter-pyspark`
  - Wait for a few seconds
  - [ ] Clic on `Copier le mot de passe` and then `Ouvrir le service`
  - [ ] On the next page, paste it in the appropriate place and `Log in`
  - You are now in your Jupyter service
- [ ] Import file named `lab1-SSPCloud.ipynb`
  - [ ] open it and follow the instructions

#### At this end of this lab

- [ ] Export your notebook
  - File > Export Notebook As ... > Export Notebook To HTML
- [ ] Delete your Jupyter service !
    
## 2. Create a Spark session

* [ ] Execute the following two cells to initialize a Spark session.

In [4]:
import os
from pyspark.sql import SparkSession

spark = (SparkSession 
         .builder
         # default url of the internally accessed Kubernetes API
         # (This Jupyter notebook service is itself a Kubernetes Pod)
         .master("k8s://https://kubernetes.default.svc:443")
         # Executors spark docker image: for simplicity reasons, this jupyter notebook is reused 
         .config("spark.kubernetes.container.image", os.environ['IMAGE_NAME'])
         # Name of the Kubernetes namespace
         .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])
         # Allocated memory to the JVM
         # Stay careful, by default, the Kubernetes pods has a higher limit which depends on other parameters.
         .config("spark.executor.memory", "4g")
         .config("spark.kubernetes.driver.pod.name", os.environ['KUBERNETES_POD_NAME'])
         # dynamic allocation configuration
         .config("spark.dynamicAllocation.enabled","true")
         .config("spark.dynamicAllocation.initialExecutors","1")
         .config("spark.dynamicAllocation.minExecutors","1")
         .config("spark.dynamicAllocation.maxExecutors","60")
         # Ratio match the number of pods to create for a given number of parallel tasks 
         # (100 parallel, ratio of 1, one aims at 100 pods, with 0.5 it would be 50 pods)
         .config("spark.dynamicAllocation.executorAllocationRatio","1")
         .config("spark.dynamicAllocation.shuffleTracking.enabled","true")
         .getOrCreate()
        )

In [5]:
spark

## 3. First steps with Spark - Data importation

Spark's main object class is the DataFrame, which is a distributed table.  
It is analogous to R's or Python (Pandas)'s data frames: 

- one row represents an observation,
- one column represents a variable.

But contrary to R or Python, Spark's DataFrames can be distributed over hundred of nodes.

Spark support multiple data formats, and multiple  ways to load them.

- data format : csv, json, parquet (an open source column oriented format)
- can read archive files
- schema detection or user defined schema. For static data, like a json file, schema detection can be use with good results.

Spark has multiple syntaxes to import data. Some are simple with no customisation, others are more complexes but you can specify options.

The simplest syntaxes to load a json or a csv file are :

```python
# JSON
json_df = spark.read.json([location of the file])
# csv
csv_df = spark.read.csv([location of the file])

```

In the future, you may consult the [Data Source documentation](https://spark.apache.org/docs/latest/sql-data-sources.html) to have the complete description of Spark's reading abilities.

The data you will use in this lab are real data from the twitter [sampled stream API](https://developer.twitter.com/en/docs/twitter-api/tweets/sampled-stream/introduction) and [filtered stream API](https://developer.twitter.com/en/docs/twitter-api/tweets/filtered-stream/introduction). The tweets folder contains more than 50 files and more than 2 million tweets. The tweets was collected between the 14/04/2021 and the 18/04/2021. The total collection time was less than 10 hours.

---

### ✍Hands-on 1  - Data importation

- [ ] Load the json file stored here : `s3a://ludo2ne/diffusion/tweets.jsonl.gz`
  - [ ] Name you data frame `df_tweet`
  - [ ] Use function `cache()` on the data frame. Caching is a performance optimization technique that allows you to persist an intermediate or final result of a computation in memory, reducing the need to recompute the data when it is accessed multiple time

  <small> ⚙️ This file is an a `JSONL` (JSON-line) format, which means that each line of it is a JSON object. A JSON object is just a Python dictionary or a JavaScript object and looks like this: `{ key1: value1, key2: ["array", "of", "many values]}`). This file has been compressed into a `GZ` archive, hence the `.jsonl.gz` ending. Also this file is not magically appearing in your S3 storage. It is hosted on one of your teacher's bucket and has been made public, so that you can access it.</small>

It's possible to load multiple file in a unique DataFrame. It's useful when you have daily files and want to process them all. It's the same syntax as the previous one, just specify a folder.

* If you meet some issue to load this file, you can load and use your own file :
  * In Onyxia, `mes fichiers`
  * Load file `tweets.jsonl.gz`
  * In Jupyter, read it using `s3a://<user_name>/tweets.jsonl.gz`


In [6]:
# DataFrame creation
df_tweet = spark.read.json("s3a://ludo2ne/diffusion/tweets.jsonl.gz")

# caching
df_tweet.cache()

DataFrame[auteur: string, contenu: string, date_creation: string, entities: struct<annotations:array<struct<end:bigint,normalized_text:string,probability:double,start:bigint,type:string>>,cashtags:array<struct<end:bigint,start:bigint,tag:string>>,hashtags:array<struct<end:bigint,start:bigint,tag:string>>,mentions:array<struct<end:bigint,id:string,start:bigint,username:string>>,urls:array<struct<description:string,display_url:string,end:bigint,expanded_url:string,images:array<struct<height:bigint,url:string,width:bigint>>,start:bigint,status:bigint,title:string,unwound_url:string,url:string>>>, hashtags: array<string>, like_count: bigint, other: struct<auteur_name:string>, reply_count: bigint, retweet_count: bigint]

## 4. Data frame basic manipulations

If DataFrames are immutable, they can however be **_transformed_** in other DataFrames, in the sense that a modified copy is returned.  
Such **transformations** include: filtering, sampling, dropping columns, selecting columns, adding new columns...

First, you can get information about the columns with:

```python
df.columns       # get the column names
df.schema        # get the column names and their respective type
df.printSchema() # same, but human-readable
```

You can select columns with the `select()` method. It takes as argument a list of column name. For example :

```python
df_with_less_columns = df\
  .select("variable3","variable_four","variable-6")

# Yes, you do need the ugly \ at the end of the line,
# if you want to chain methods between lines in Python
```

You can get nested columns easily with :

```python
df.select("parentField.nestedField")
```

To filter data you could use the `filter()` method. It take as input an expression that gets evaluated for each observation and should return a boolean. Sampling is performed with the `sample()` method. For example :

```python
df_with_less_rows = df\
  .sample(fraction=0.001)\
  .filter(df.variable1=="value")\
  .show(10)
```

As said before your data are distributed over multiple nodes (executors) and data inside a node are split into partitions. Then each transformations will be run in parallel. They are called *narrow transformation* For example, to sample a DataFrame, Spark sample every partitions in parallel because sample all partition produce the sample DataFrame. For some transformations, like `groupBy()` it's impossible, and it's cannot be run in parallel.

<img src="img/spark_exemple2_pipeline.png" style="zoom:50%;" />

<!-- take() collect() limit() first() show() -->
<!-- lien vers la doc https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#dataframe-apis -->

### Lazy evaluation

This is because Spark has what is known as **lazy evaluation**, in the sense that it will wait as much as it can before performing the actual computation. Said otherwise, when you run an instruction such as:

```python
tweet_author_hashtags = df_tweet_big.select("auteur","hashtags")
```

... you are not executing anything! Rather, you are building an **execution plan**, to be realised later.

Spark is quite extreme in its laziness, since only a handful of methods called **actions**, by opposition to **transformations**, will trigger an execution. The most notable are:

1. `collect()`, explicitly asking Spark to fetch the resulting rows instead of to lazily wait for more instructions,
2. `take(n)`, asking for `n` first rows
3. `first()`, an alias for `take(1)`
4. `show()` and `show(n)`, human-friendly alternatives
5. `count()`, asking for the numbers of rows
6. all the "write" methods (write on file, write to database), see [here](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#input-and-output) for the list

**This has advantages:** on huge data, you don't want to accidently perform a computation that is not needed. Also, Spark can optimize each **stage** of the execution in regard to what comes next. For instance, filters will be executed as early as possible, since it diminishes the number of rows on which to perform later operations. On the contrary, joins are very computation-intense and will be executed as late as possible. The resulting **execution plan** consists in a **directed acyclic graph** (DAG) that contains the tree of all required actions for a specific computation, ordered in the most effective fashion.

**This has also drawbacks.** Since the computation is optimized for the end result, the intermediate stages are discarded by default. So if you need a DataFrame multiple times, you have to cache it in memory because if you don't Spark will recompute it every single time. 

---

### ✍Hands-on 2 - Data frame basic manipulations

- [ ] How many rows have your two DataFrame ?

In [4]:
df_tweet.count()

100000

- [ ] Display columns names and then the schema

In [9]:
df_tweet.columns

['auteur',
 'contenu',
 'date_creation',
 'entities',
 'hashtags',
 'like_count',
 'other',
 'reply_count',
 'retweet_count']

In [7]:
df_tweet.printSchema()

root
 |-- auteur: string (nullable = true)
 |-- contenu: string (nullable = true)
 |-- date_creation: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- annotations: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- end: long (nullable = true)
 |    |    |    |-- normalized_text: string (nullable = true)
 |    |    |    |-- probability: double (nullable = true)
 |    |    |    |-- start: long (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |-- cashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- end: long (nullable = true)
 |    |    |    |-- start: long (nullable = true)
 |    |    |    |-- tag: string (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- end: long (nullable = true)
 |    |    |    |-- start: long (nullable = true)
 |    |    |    |-- tag:

- [ ] Display 10 rows of df_tweet

In [11]:
df_tweet.show(10)

+---------------+-------------------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+
|         auteur|                        contenu|       date_creation|            entities|hashtags|like_count|               other|reply_count|retweet_count|
+---------------+-------------------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+
|    xwendystian|           RT @karlarboledas...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|            {W🚶‍♀️}|          0|          962|
|    suoicilataR|           RT @sooky_co: SLA...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|               {Rat}|          0|         3849|
|        SatsuOW|           RT @kusujinn: 👻?...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|             {Satsu}|          0|         9787|
|   birashoboka7|           @Kitona_gold You ...

- [ ] Sample `df_tweet` and keep only 10% of it.
  - Create a new DataFrame named `df_tweet_sampled`.
  - If computations take too long on the full DataFrame, use this one instead or add a sample transformation in your expression. 

In [13]:
df_tweet_sampled = df_tweet\
  .sample(fraction=0.1)

df_tweet_sampled.count()

10145

- [ ] Define a DataFrame `tweet_author_hashtags` with only the `auteur` and `hashtags` columns
  - [ ] Then display 5 rows

In [14]:
tweet_author_hashtags = df_tweet.select("auteur", "hashtags")

tweet_author_hashtags.show(5)

+------------+--------+
|      auteur|hashtags|
+------------+--------+
| xwendystian|      []|
| suoicilataR|      []|
|     SatsuOW|      []|
|birashoboka7|      []|
| TheRealCzia|      []|
+------------+--------+
only showing top 5 rows



- [ ] Print 5 lines of a **df_tweet** with only the `auteur`, `mentions`, and `urls` columns.
  - `mentions` and `urls` are both nested columns in `entities`

In [15]:
df_tweet\
  .select("auteur", 
          "entities.mentions", 
          "entities.urls")\
  .show(5)

+------------+--------------------+--------------------+
|      auteur|            mentions|                urls|
+------------+--------------------+--------------------+
| xwendystian|[{17, 11851291015...|                NULL|
| suoicilataR|[{12, 14533927358...|[{NULL, pic.twitt...|
|     SatsuOW|[{12, 11935477705...|[{NULL, pic.twitt...|
|birashoboka7|[{12, 14860437391...|                NULL|
| TheRealCzia|                NULL|                NULL|
+------------+--------------------+--------------------+
only showing top 5 rows



- [ ] Filter **df_tweet** and keep only tweets with more than 1 like.
  - [ ] Display only `auteur`, `contenu` and `like_count`
  - [ ] Print 10 lines

In [16]:
df_tweet\
  .filter(df_tweet.like_count >= 1)\
  .show(10)

+---------------+------------------------------------+--------------------+--------------------+---------+----------+--------------------------------+-----------+-------------+
|         auteur|                             contenu|       date_creation|            entities| hashtags|like_count|                           other|reply_count|retweet_count|
+---------------+------------------------------------+--------------------+--------------------+---------+----------+--------------------------------+-----------+-------------+
|      DZero_SYS|                @blueberry_lin 🖥...|2022-03-24T14:59:...|{NULL, NULL, NULL...|       []|         1|               {포켓워치 시스템}|          0|            0|
|exusial_Penguin|            @Hano_Braves へぇ、い...|2022-03-24T14:59:...|{NULL, NULL, NULL...|       []|         1|                      {エクシア}|          0|            0|
|      queend236|                @dayytonerr I try...|2022-03-24T14:59:...|{NULL, NULL, NULL...|       []|         1|            {u

## 5. Basic DataFrame column manipulation 

You can add/update/rename column of a DataFrame using **spark** :

- Drop : `df.drop(columnName : str )`
- Rename : `df.withColumnRenamed(oldName : str, newName : str)`
- Add/update : `df.withColumn(columnName : str, columnExpression)` 

For example

```python
df_tweet\
  .withColumn(                                        # computes new variable
    "like_rt_ratio",                                  # like_rt_ratio "OVERCONFIDENCE"
    df_tweet.like_count /df_tweet.retweet_count)
```

See **[here](https://spark.apache.org/docs/3.1.1/api/python/reference/pyspark.sql.html#functions)** for the list of all functions available in an expression.

### ✍Hands-on 3 - Basic DataFrame column manipulation  

- [ ] Define a DataFrame with a column names `interaction_count` named `df_tweet_interaction_count`
  - This column is the sum of `like_count`, `reply_count` and `retweet_count`.

In [17]:
df_tweet_interaction_count = df_tweet\
  .drop("other")\
  .filter(df_tweet.like_count >= 1)\
  .filter(df_tweet.retweet_count >= 1)\
  .withColumn(
    "interaction_count",
    df_tweet.like_count + df_tweet.reply_count + df_tweet.retweet_count)

df_tweet_interaction_count\
  .show(10)

+--------------+-------------------------------------+--------------------+---------------------------+--------------------------------+----------+-----------+-------------+-----------------+
|        auteur|                              contenu|       date_creation|                   entities|                        hashtags|like_count|reply_count|retweet_count|interaction_count|
+--------------+-------------------------------------+--------------------+---------------------------+--------------------------------+----------+-----------+-------------+-----------------+
|      Unif1992|                 kalıplaşmış #beyo...|2022-03-24T14:59:...|       {NULL, NULL, [{20...|                       [beyoğlu]|         1|          0|            2|                3|
|    moonknight|                 The wait is almos...|2022-03-24T15:00:...|       {[{40, Marvel Stu...|                    [MoonKnight]|        16|          0|            2|               18|
|         B04Zw|                #PayPay\

- [ ] Update the DataFrame you imported at the beginning of this lab and drop the `other` column

In [18]:
df_tweet\
  .drop("other")\
  .show(10)

+---------------+-------------------------------+--------------------+--------------------+--------+----------+-----------+-------------+
|         auteur|                        contenu|       date_creation|            entities|hashtags|like_count|reply_count|retweet_count|
+---------------+-------------------------------+--------------------+--------------------+--------+----------+-----------+-------------+
|    xwendystian|           RT @karlarboledas...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|          0|          962|
|    suoicilataR|           RT @sooky_co: SLA...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|          0|         3849|
|        SatsuOW|           RT @kusujinn: 👻?...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|          0|         9787|
|   birashoboka7|           @Kitona_gold You ...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|          0|            0|
|    TheRealCzia|           nagugul

## 6. Advance DataFrame column manipulation 

### Array manipulation

Some columns often contain arrays (lists) of values instead of just one value. This may seem surprising but this actually quite natural.  
For instance, you may create an array of words from a text, or generate a list of random numbers for each observation, etc.

You may **create array of values** with:

- `split(text : string, delimiter : string)`, turning a text into an array of strings

You may **use array of values** with:

- `size(array : Array)`, getting the number of elements
- `array_contains(inputArray : Array, value : any)`, checking if some value appears
- `explode(array : Array)`, unnesting an array and duplicating other values. For instance if you use `explode()` over the hashtags value of this DataFrame:

  | Auteur | Contenu                             | Hashtags         |
  | ------ | ----------------------------------- | ---------------- |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] |
  | Alice  | Just finished #MHrise, best MH ever | [MHrise]         |

  You will get :

  | Auteur | Contenu                             | Hashtags         | Hashtag |
  | ------ | ----------------------------------- | ---------------- | ------- |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] | Spark   |
  | Bob    | I love #Spark and #bigdata          | [Spark, bigdata] | bigdata |
  | Alice  | Just finished #MHrise, best MH ever | [MHrise]         | MHrise  |

  

All this functions must be imported first :

```python
from pyspark.sql.functions import split, explode, size, array_contains
```

Do not forget, to create a new column, you should use `withColumn()`. For example : 

```python
df.withColumn("new column", explode("array"))
```

#### ✍Hands-on 4 - Array manipulation 

- [ ] Keep all the tweets with hashtags and for each remaining line, split the hashtag text into an array of hashtags

In [19]:
from pyspark.sql.functions import split, explode, size, array_contains

df_tweet\
  .filter(size("hashtags") > 0)\
  .withColumn(
    "hashtag_exploded",
    explode("hashtags"))\
  .show(10)

+------------+-----------------------+--------------------+----------------------------+---------------------+----------+--------------------+-----------+-------------+-----------------+
|      auteur|                contenu|       date_creation|                    entities|             hashtags|like_count|               other|reply_count|retweet_count| hashtag_exploded|
+------------+-----------------------+--------------------+----------------------------+---------------------+----------+--------------------+-----------+-------------+-----------------+
|     GIimoto|RT @aabbeecc: 日本コ...|2022-03-24T14:59:...|{[{21, 日本コカ・コーラ, ...|[utamaru, コークオン]|         0|       {g.orchestra}|          0|            4|          utamaru|
|     GIimoto|RT @aabbeecc: 日本コ...|2022-03-24T14:59:...|{[{21, 日本コカ・コーラ, ...|[utamaru, コークオン]|         0|       {g.orchestra}|          0|            4|       コークオン|
|  MEWWhiskey|   📍เปิดหาร nct dre...|2022-03-24T14:59:...|        {NULL, NULL, [{14...| [ตลาดนัดnctdre

- [ ] Create a new column with the number of words of the `contenu` column. (Use `split()` + `size()`)

In [20]:
df_tweet\
  .withColumn("word_count", 
              size(split("contenu", " ")))\
  .show(5)

+------------+--------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+----------+
|      auteur|             contenu|       date_creation|            entities|hashtags|like_count|               other|reply_count|retweet_count|word_count|
+------------+--------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+----------+
| xwendystian|RT @karlarboledas...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|            {W🚶‍♀️}|          0|          962|         7|
| suoicilataR|RT @sooky_co: SLA...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|               {Rat}|          0|         3849|         7|
|     SatsuOW|RT @kusujinn: 👻?...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|             {Satsu}|          0|         9787|         4|
|birashoboka7|@Kitona_gold You ...|2022-03-24T14:59:...|{NULL, NUL

- [ ] Count how many tweet contain the `Ukraine` hashtag (use the `count()` action)

In [21]:
df_tweet\
  .filter(array_contains("hashtags", "Ukraine"))\
  .count()

35

### User defined function

For more very specific column manipulation you will need Spark's `udf()` function (*User Defined Function*). It can be useful if Spark does not provide a feature you want. But Spark is a popular and active project, so before coding an udf, go check the documentation. For instance for natural language processing, Spark already has some [functions](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.feature.Tokenizer.html#pyspark.ml.feature.Tokenizer). Last things, python udf can lead to performance issues (see https://stackoverflow.com/a/38297050) and learning a little bit of scala or java can be a good idea.

For example :

```python
# !!!! DOES NOT WORK !!!!
def to_lower_case(string):
	return string.lower()
	
df.withColumn("tweet_lower_case", to_lower_case(df.contenu))
```

will just crash. Keep in mind that Spark is a distributed system, and that Python is only installed on the central node, as a convenience to let you execute instructions on the executor nodes. But by default, pure Python functions can only be executed where Python is installed! We need `udf()` to enable Spark to send Python instructions to the worker nodes.

Let us see how it is done :

```python
# imports
from pyspark.sql.functions import udf
from pyspark.sql.functions import explode
from pyspark.sql.types import StringType

# pure python functions
def to_lower_case(string):
    return string.lower()

# user defined function(we use a lambda function to create the udf)
to_lower_case_udf = udf(
    lambda x: to_lower_case(x), StringType()
)

# df manipulation
df_tweet_small\
  .select("auteur","hashtags")\
  .filter("size(hashtags)!=0")\
  .withColumn("hashtag", explode("hashtags"))\
  .withColumn("hashtag", to_lower_case_udf("hashtag")).show(10)
```

---

#### ✍Hands-on 5 - User defined function 

- [ ] Create an user defined function that counts how many words a tweet contains.
  - your function will return an `IntegerType` and not a `StringType`

In [22]:
# imports
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# pure python functions
def word_count(string):
    return len(string.split(" "))

# user definid function
word_count_udf = udf(
    lambda x: word_count(x), IntegerType()
)

# df manipulation
df_tweet\
  .withColumn("word_count", 
              word_count_udf("contenu"))\
  .show(10)

+---------------+-------------------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+----------+
|         auteur|                        contenu|       date_creation|            entities|hashtags|like_count|               other|reply_count|retweet_count|word_count|
+---------------+-------------------------------+--------------------+--------------------+--------+----------+--------------------+-----------+-------------+----------+
|    xwendystian|           RT @karlarboledas...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|            {W🚶‍♀️}|          0|          962|         7|
|    suoicilataR|           RT @sooky_co: SLA...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|               {Rat}|          0|         3849|         7|
|        SatsuOW|           RT @kusujinn: 👻?...|2022-03-24T14:59:...|{NULL, NULL, NULL...|      []|         0|             {Satsu}|          0|        

## 7. Aggregation functions

Spark offer a variety of aggregation functions :

- `count(column : string)` will count every not null value of the specify column. You cant use `count(1)` of `count("*")` to count every line (even row with only null values)

- `countDisctinct(column : string)` and `approx_count_distinct(column : string, percent_error: float)`. If the exact number is irrelevant, `approx_count_distinct()`should be preferred.

  Counting distinct elements cannot be done in parallel, and need a lot data transfer. But if you only need an approximation, there is a algorithm, named hyper-log-log (more info [here](https://databricks.com/fr/blog/2016/05/19/approximate-algorithms-in-apache-spark-hyperloglog-and-quantiles.html)) that can be parallelized. 

  ```python
  from pyspark.sql.functions import count, countDistinct, approx_count_distinct
  
  df.select(count("col1")).show()
  df.select(countDistinct("col1")).show()
  df.select(approx_count_distinct("col1"), 0.1).show()
  ```

- You have access to all other common functions `min()`, `max()`, `first()`, `last()`, `sum()`, `sumDistinct()`, `avg()` etc (you should import them first `from pyspark.sql.functions import min, max, avg, first, last, sum, sumDistinct`) 

---

### ✍Hands-on 6 - Aggregation functions

- [ ] What are the min, max, average of `interaction_count` (use `df_tweet_interaction_count` created earlier)
  - don't forget to import the required functions

In [23]:
from pyspark.sql.functions import min, max, avg, first, last, sum, sumDistinct

df_tweet_interaction_count\
  .select(min("interaction_count"),
          max("interaction_count"),
          avg("interaction_count"))\
  .first()

Row(min(interaction_count)=2, max(interaction_count)=3130, avg(interaction_count)=56.883720930232556)

- [ ] How many tweets have hashtags ?
  - [ ] Distinct hashtags ?
  - [ ] Try the approximative count with 0.1 and 0.01 as maximum estimation error allowed.

In [24]:
from pyspark.sql.functions import count, countDistinct, approx_count_distinct

df_tweet\
  .select(count("hashtags"), 
          countDistinct("hashtags"), 
          approx_count_distinct("hashtags", 0.1), 
          approx_count_distinct("hashtags",0.01))\
  .show()

+---------------+------------------------+-------------------------------+-------------------------------+
|count(hashtags)|count(DISTINCT hashtags)|approx_count_distinct(hashtags)|approx_count_distinct(hashtags)|
+---------------+------------------------+-------------------------------+-------------------------------+
|         100000|                    9320|                           7826|                           9342|
+---------------+------------------------+-------------------------------+-------------------------------+



## 8. Grouping functions

Like SQL you can group row by a criteria with Spark. Just use the `groupBy(column : string)` method. Then you can compute some aggregation over those groups.

```python
df\
  .groupBy("col1")\
  .agg(count("col2").alias("quantity"))\           # alias is use to specify the name of the new column
  .show() 
```

The `agg()` method can take multiples argument to compute multiple aggregation at once.

```python
df\
  .groupBy("col1")\
  .agg(count("col2").alias("quantity"), 
       min("col2").alias("min"), 
       avg("col3").alias("avg3"))\
  .show()
```

Aggregation and grouping transformations work differently than the previous method like `filter()`, `select()`, `withColumn()` etc. Those transformations cannot be run over each partitions in parallel, and need to transfer data between partitions and executors.  They are called "wide transformations"

<img src="img/spark_exemple2_pipeline.png" style="zoom:50%;" />

### ✍Hands-on 7 - Grouping functions

- [ ] Compute a daframe with the min, max and average retweet of each `auteur`.
  - [ ] Then order it by the max number of retweet in descending order by .
  - To do that you can use the following syntax

  ```python
  from pyspark.sql.functions import desc
  df.orderBy(desc("col"))
  ```

In [ ]:
from pyspark.sql.functions import desc

df_tweet\
  .groupBy("auteur")\
  .agg(min("retweet_count").alias("min_RT"), 
       max("retweet_count").alias("max_RT"), 
       avg("retweet_count").alias("avg_RT"))\
  .orderBy(desc("max_RT"))\
  .show(5)

## 9. Spark SQL

Spark understand SQL statement. It's not a hack nor a workaround to use SQL in Spark, it's one a the more powerful feature in Spark. To use SQL you will need :

1. Register a view pointing to your DataFrame

   ```python
   my_df.createOrReplaceTempView(viewName : str)
   ```

2. Use the sql function

   ```python
   spark.sql("""
   Your SQL statement
   """)
   ```

   You could manipulate every registered DataFrame by their view name with plain SQL.

In fact you can do most of this tutorial without any knowledge in PySpark nor Spark.  
Many things can only be done in Spark if you know SQL and how to use it in Spark. 

### ✍Hands-on 8 - Spark SQL

- [ ] How many tweets have hashtags ?
  - [ ] Distinct hashtags ? 

In [ ]:
df_tweet\
  .select("contenu", "hashtags")\
  .createOrReplaceTempView("view_hashtag_content")

spark.sql("""
SELECT COUNT(*), 
       COUNT(DISTINCT(contenu))
  FROM view_hashtag_content
 WHERE size(hashtags) > 0
""")\
  .show(5)

- [ ] Compute a dataframe with the min, max and average retweet of each `auteur` using Spark SQL

In [ ]:
df_tweet.createOrReplaceTempView("view_tweet")

spark.sql("""
SELECT min(retweet_count), 
       max(retweet_count), 
       avg(retweet_count)
  FROM view_tweet
 GROUP BY auteur
 ORDER BY MAX(retweet_count) DESC
""")\
  .show(5)

# 10. Delete the Jupyter-pyspark service

* [ ] Export your notebook
  - Right clic and Download (.ipynb)
  - File > Save and Export Notebook > HTML
* https://datalab.sspcloud.fr/my-services
